In [20]:
import asyncio
import math
import time
import numpy as np
import pandas as pd
from distutils.ccompiler import gen_preprocess_options
import xml.etree.cElementTree as ET
from threading import Thread

import qtm

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.mem import MemoryElement
from cflib.crazyflie.mem import Poly4D
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.utils import uri_helper
from cflib.positioning.motion_commander import MotionCommander

In [21]:
# URI to the crazyflie to connect to 
uri = uri_helper.uri_from_env(default='radio://0/80/2M/E7E7E7E7E7')

#The name of the rigid Body in QTM that represents the Crazyflie
rigid_body_name = 'cfe7'

#True: send position and orientation; False: send position only
send_full_pose = False

In [22]:
DEFAULT_HEIGHT = 0.5
TS = 0.05
KP = 2.3
KD = 0.557
FACTOR  = 2
UD = 0.06*FACTOR
RX = 0.07*FACTOR
RY = 0.07*FACTOR
ALONGZ = False
SIMULATING_TIME = 15


In [23]:
class QtmWrapper(Thread):
    def __init__(self, body_name):
        Thread.__init__(self)

        self.body_name = body_name
        self.on_pose = None
        self.connection = None
        self.qtm_6DoF_labels = []
        self._stay_open = True
        self.pose = []

        self.start()

    def close(self):
        self._stay_open = False
        self.join()

    def run(self):
        asyncio.run(self._life_cycle())

    async def _life_cycle(self):
        await self._connect()
        while (self._stay_open):
            await asyncio.sleep(1)
        await self.close()

    async def _connect(self):
        host = 'IP_Address'
        print('Connecting to QTM on' + host)
        self.connection = await qtm.connect(host)

        params = await self.connection.get_parameters(parameters=['6d'])
        xml = ET.fromstring(params)
        self.qtm_6DoF_labels = [label.text.strip() for index, label in enumerate(xml.findall('*/Body/Name'))]

        await self.connection.stream_frames(
            components=['6D', '6dEuler'], on_packet=self._on_packet)
        
    async def _discover(self):
        async for qtm_instance in qtm.Discover('0.0.0.0'):
            return qtm_instance
        
    def _on_packet(self, packet):

        header, bodies = packet.get_6d()
        header_eulor, bodies_eulor = packet.get_6d_euler()

        if bodies is None :
            return
        
        if self.body_name not in self.qtm_6DoF_labels:
            print('Body' + self.body_name + 'not found :(')
        else: 
            index = self.qtm_6DoF_labels.index(self.body_name)
            temp_cf_pos = bodies[index]
            self.pose = bodies[index]
            self.pose = bodies.euler[index]
            x = temp_cf_pos[0][0] / 1000
            y = temp_cf_pos[0][1] / 1000
            z = temp_cf_pos[0][2] / 1000

            r = temp_cf_pos[1].matrix
            rot = [
                [r[0], r[3], r[6]],
                [r[1], r[4], r[7]],
                [r[2], r[5], r[8]],
            ]

            if self.on_pose:
                # Make sure we got a position
                if math.isnan(x):
                    return
                self.on_pose([x, y, z, rot])

    async def _close(self):
        await self.connection.stream_frames_stop()
        self.connection.disconnect()


In [24]:
def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001

    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            # print("{} {} {}".
            #       format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break



In [25]:
def _sqrt(a):
    """
    There might be rounding errors making 'a' slightly negative.
    Make sure we don't throw an exception.
    """
    if a < 0.0:
        return 0.0
    return math.sqrt(a)


In [26]:
def send_extpose_rot_matrix(cf, x, y, z, rot):
    """
    Send the current Crazyflie X, Y, Z position and attitude as a (3x3)
    rotaton matrix. This is going to be forwarded to the Crazyflie's
    position estimator.
    """
    qw = _sqrt(1 + rot[0][0] + rot[1][1] + rot[2][2]) / 2
    qx = _sqrt(1 + rot[0][0] - rot[1][1] - rot[2][2]) / 2
    qy = _sqrt(1 - rot[0][0] + rot[1][1] - rot[2][2]) / 2
    qz = _sqrt(1 - rot[0][0] - rot[1][1] + rot[2][2]) / 2

    # Normalize the quaternion
    ql = math.sqrt(qx ** 2 + qy ** 2 + qz ** 2 + qw ** 2)

    if send_full_pose:
        cf.extpos.send_extpose(x, y, z, qx / ql, qy / ql, qz / ql, qw / ql)
    else:
        cf.extpos.send_extpos(x, y, z)


In [27]:
def take_off(cf, position,take_off_time=1,sleep_time=0.1):
    # take_off_time = 1.0
    # sleep_time = 0.1
    steps = int(take_off_time / sleep_time)
    vz = position[2] / take_off_time

    print(vz)

    for i in range(steps):
        cf.commander.send_velocity_world_setpoint(0, 0, vz, 0)
        time.sleep(sleep_time)


In [28]:



def land(cf, position):
    landing_time = 1.0
    sleep_time = 0.1
    steps = int(landing_time / sleep_time)
    vz = -position[2] / landing_time

    print(vz)

    for _ in range(steps):
        cf.commander.send_velocity_world_setpoint(0, 0, vz, 0)
        time.sleep(sleep_time)

    cf.commander.send_stop_setpoint()
    # Make sure that the last packet leaves before the link is closed
    # since the message queue is not flushed before closing
    time.sleep(0.1)


In [29]:
def reset_estimator(cf):
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')

    # time.sleep(1)
    wait_for_position_estimator(cf)


In [30]:
def activate_kalman_estimator(cf):
    cf.param.set_value('stabilizer.estimator', '2')

    # Set the std deviation for the quaternion data pushed into the
    # kalman filter. The default value seems to be a bit too low.
    cf.param.set_value('locSrv.extQuatStdDev', 0.06)


In [31]:
def BS_run_sequence_velocity(cf, sequence):

    for vs in sequence:
        print('Velocity {}'.format(vs))

        vx = vs[0] 
        vy = vs[1] 
        vz = vs[2] 
        vyaw = 0

        for i in range(10):
            # cf.commander.send_position_setpoint(x, y, z, yaw)
            # cf.commander.send_velocity_world_setpoint(vx, vy, vz, vyaw)
            cf.commander.send_hover_setpoint(vx, vy, vyaw,DEFAULT_HEIGHT)
            time.sleep(0.1)
        time.sleep(1)

        
    cf.commander.send_stop_setpoint()
    # Make sure that the last packet leaves before the link is closed
    # since the message queue is not flushed before closing
    time.sleep(0.1)


In [32]:
def prepare_desired_trajactory(current_pose=[0,0,0],SimulationTime=5,Ts=0.1,track='line',ud=0.12,factor = 2,offsety=0):
    
    DesiredTime=SimulationTime+1
    FloatState = current_pose
    timeseries = np.arange(0,DesiredTime,Ts)
    Xd = np.zeros(len(timeseries))
    Yd = np.zeros(len(timeseries))
    Desired = np.zeros((len(timeseries),3))

    Xd[0]=FloatState[0]

    for bs, t in enumerate(timeseries[:-1]):
        
        if track == 'line':
            Xd[bs+1]=Xd[bs]+ud*Ts
        elif track == 'sine':
            Xd[bs+1]=Xd[bs]+ud*Ts/np.sqrt(1+np.cos(factor*Xd[bs])**2)


    Yd[0]=FloatState[1];
    for bs, t in enumerate(timeseries[:-1]):
        if track == 'line':
            Yd[bs+1]=Yd[bs]
        elif track == 'sine':
            Yd[bs+1]=Yd[bs]+ud*np.cos(factor*Xd[bs])*Ts/np.sqrt(1+np.cos(factor*Xd[bs])**2)


    Desired[:,0]=Xd;
    Desired[:,1]=Yd;
    Desired[:,2]=np.zeros(len(timeseries));

    if track == 'sine':
        Psaid = np.zeros(len(timeseries))
        Psaid[0]=current_pose[2]
        for bs, t in enumerate(timeseries[:-1]):
            Psaid[bs+1]=np.arctan(np.cos(factor*Xd[bs]));
        Desired[:,2] = Psaid;

    Desired[:,1] = Desired[:,1]+offsety;

    return Desired


In [ ]:
if __name__ == '__main__':
    cflib.crtp.init_drivers()

    # Connect to QTM
    qtm_wrapper = QtmWrapper(rigid_body_name)

    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        cf = scf.cf
        trajectory_id = 1

        # Set up a callback to handle data from QTM
        qtm_wrapper.on_pose = lambda pose: send_extpose_rot_matrix(
            cf, pose[0], pose[1], pose[2], pose[3])

        activate_kalman_estimator(cf)
        reset_estimator(cf)

        current_x = qtm_wrapper.pose[0][0]/1000.
        current_y = qtm_wrapper.pose[0][1]/1000.
        current_z = qtm_wrapper.pose[0][2]/1000.
        current_yaw = qtm_wrapper.pose[1][0]

        if ALONGZ:
            offsety = 1.3
        else:
            offsety = 0

        print('current_x',current_x,'current_z',current_z)
        current_pose = np.array([current_x,current_z,current_yaw])
        Desired = prepare_desired_trajactory(current_pose=current_pose,
                                             SimulationTime=SIMULATING_TIME,
                                             ud=UD,
                                             track='sine',
                                             offsety=offsety)

        if ALONGZ:
            offset = 0.5-np.min(Desired[:,1])
            Desired[:,1] = Desired[:,1] +offset

        state_sim = np.array([current_pose])
        take_off(cf,
                 [current_x, current_y, Desired[0,1], current_yaw],
                 take_off_time=1)

        xe_last = 0
        ye_last = 0
        # states_debug = np.empty((1,15))
        for ii, desired in enumerate(Desired):
            start_time = time.time()

            current_x = qtm_wrapper.pose[0][0]/1000.
            current_y = qtm_wrapper.pose[0][1]/1000.
            current_z = qtm_wrapper.pose[0][2]/1000.
            current_yaw = qtm_wrapper.pose[1][0]
            if ALONGZ:
                current_pose = np.array([[current_x,current_z,current_yaw]])
            else:
                current_pose = np.array([[current_x,current_y,current_yaw]])

            state_sim  = np.append(state_sim,current_pose,0)
            xe_in    =   state_sim[-1,0]-Desired[ii,0]
            ye_in    =   state_sim[-1,1]-Desired[ii,1]
            ang_in   =   Desired[ii,2]

            ang_temp=   -ang_in;
            xe_o    =   xe_in*np.cos(ang_temp) - ye_in*np.sin(ang_temp);
            ye_o    =   xe_in*np.sin(ang_temp) + ye_in*np.cos(ang_temp);

            xedt    =   (xe_o-xe_last)/TS;
            yedt    =   (ye_o-ye_last)/TS;

            vx      =   -(KP*xe_o+KD*xedt)*np.exp(-xe_o**2/(2*RX**2))
            vy      =   -(KP*ye_o+KD*yedt)*np.exp(-ye_o**2/(2*RY**2))

            # vx      =   -(KP*xe_o+KD*xedt)
            # vy      =   -(KP*ye_o+KD*yedt)

            # print(xe_o,vx)
            
            # vx_o = vx
            # vy_o = vy

            vx_o = np.sign(vx)*np.min([np.abs(vx),1])
            vy_o = np.sign(vy)*np.min([np.abs(vy),1])
            vyaw = 0

            
            # vyaw = np.sign(vyaw)*np.min([np.abs(vyaw),30])

            # ang_out =   -ang_temp;
            # vx_o    =   vx*np.cos(ang_out) - vy*np.sin(ang_out)
            # vy_o    =   vx*np.sin(ang_out) + vy*np.cos(ang_out)
            # vyaw    =  (np.rad2deg(np.arctan(vy_o/vx_o)) - current_yaw)/TS

            # vx_o = np.sign(vx_o)*np.min([np.abs(vx_o),0.5])
            # vy_o = np.sign(vy_o)*np.min([np.abs(vy_o),0.5])
            # vyaw = np.sign(vyaw)*np.min([np.abs(vyaw),30])
            # print(vx_o, vy_o, vyaw)
            if ALONGZ:
                cf.commander.send_velocity_world_setpoint(vx_o, 0, vy_o, vyaw)
            else:
                cf.commander.send_hover_setpoint(vx_o, vy_o, vyaw, DEFAULT_HEIGHT)

            xe_last = np.copy(xe_o)
            ye_last = np.copy(ye_o)

            states_temp = [[desired[0],desired[1],current_x,current_y,xe_in,ye_in,ang_in,xe_o,ye_o,xedt,yedt,vx,vy,vx_o,vy_o]]
            if ii == 0:
                states_debug = states_temp
            else:
                states_debug = np.append(states_debug,states_temp,0)
            if TS-(time.time() - start_time)>0:
                time.sleep(TS-(time.time() - start_time))

    land(cf,[current_x,current_y,current_z])

    indexes = ['desired_x','desired_y','current_x','current_y','xe_in','ye_in','ang_in','xe_o','ye_o','xedt','yedt','vx','vy','vx_o','vy_o']
    df = pd.DataFrame(data=states_debug,columns=indexes)
    df.to_csv('./debug.csv')
    qtm_wrapper.close()


2023-07-05 14:50:16,345 - cflib.crazyflie - INFO - Callback->Connection initialized[radio://0/80/2M/E7E7E7E7E7]
2023-07-05 14:50:16,368 - cflib.crazyflie - ERROR - Couldn't load link driver: Cannot find a Crazyradio Dongle

Traceback (most recent call last):
  File "C:\Users\sbhar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cflib\drivers\crazyradio.py", line 124, in __init__
    device = _find_devices()[devid]
  File "C:\Users\sbhar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cflib\drivers\crazyradio.py", line 79, in _find_devices
    devices = usb.core.find(idVendor=0x1915, idProduct=0x7777, find_all=1,
  File "C:\Users\sbhar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\usb\core.py", line 1309, in find
    raise NoBackendError('No backend available')

Connecting to QTM onIP_Address


Exception: Couldn't load link driver: Cannot find a Crazyradio Dongle

Traceback (most recent call last):
  File "C:\Users\sbhar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cflib\drivers\crazyradio.py", line 124, in __init__
    device = _find_devices()[devid]
  File "C:\Users\sbhar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cflib\drivers\crazyradio.py", line 79, in _find_devices
    devices = usb.core.find(idVendor=0x1915, idProduct=0x7777, find_all=1,
  File "C:\Users\sbhar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\usb\core.py", line 1309, in find
    raise NoBackendError('No backend available')
usb.core.NoBackendError: No backend available

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
 

2023-07-05 14:50:17,367 - qtm - ERROR - [Errno 11001] getaddrinfo failed
Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\sbhar\AppData\Local\Temp\ipykernel_3756\1164677926.py", line 19, in run
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\asyncio\runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\asyncio\base_events.py", line 649, in run_until_complete
    return future.result()
  File "C:\Users\sbhar\AppData\Local\Temp\ipykernel_3756\1164677926.py", line 22, in _life_cycle
  File "C:\Users\sbhar\AppData\Local\Temp\ipykernel_3756\1164677926.py", line 32, in _connect
AttributeError: 'NoneType' obje